# VacationPy
----

In [22]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

ModuleNotFoundError: No module named 'gmaps'

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [23]:
city_df = pd.read_csv("output_data/cities.csv")
city_df.head()

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,grand river south east,38.8755,-109.5012,79.00,8,1,5.75,US,1617405174
1,1,castro,-24.7911,-50.0119,58.33,82,4,3.27,BR,1617405174
2,2,airai,-8.9266,125.4092,61.59,95,100,3.76,TL,1617405174
3,3,constitucion,-35.3333,-72.4167,57.43,83,25,13.58,CL,1617405174
4,4,fare,-16.7000,-151.0167,81.81,69,0,1.54,PF,1617405175


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [24]:
gmaps.configure(api_key=g_key)

NameError: name 'gmaps' is not defined

In [25]:
locations = city_df[["Lat", "Lng"]]
humidity = city_df["Humidity"]
fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, max_intensity=300, point_radius=5, dissipating=False)
fig.add_layer(heat_layer)

NameError: name 'gmaps' is not defined

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [26]:
narrow_df = city_df.loc[(city_df["Max Temp"] < 80) & (city_df["Max Temp"] > 70) \
                            & (city_df["Wind Speed"] < 10) \
                            & (city_df["Cloudiness"] == 0)].dropna()
narrow_df

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
28,28,pisco,-13.7000,-76.2167,71.60,73,0,8.05,PE,1617405182
67,67,buraydah,26.3260,43.9750,71.60,26,0,9.22,SA,1617405195
81,81,cayenne,4.9333,-52.3333,78.01,94,0,1.14,GF,1617405200
95,95,tessalit,20.1986,1.0114,77.58,8,0,9.42,ML,1617405204
161,161,bela,25.9333,81.9833,71.13,9,0,5.55,IN,1617405225
195,195,salalah,17.0151,54.0924,75.20,73,0,4.61,OM,1617405235
209,209,lima,-12.0432,-77.0282,73.99,68,0,8.05,PE,1617404967
215,215,quelimane,-17.8786,36.8883,75.20,88,0,0.87,MZ,1617405240
358,358,machaneng,-23.1862,27.4886,70.39,44,0,3.69,BW,1617405075
362,362,qiongshan,20.0058,110.3542,75.20,94,0,6.71,CN,1617405359


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [27]:
hotel_df = narrow_df[["City", "Country", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""

hotel_df

,City,Country,Lat,Lng,Hotel Name
28,pisco,PE,-13.7000,-76.2167,
67,buraydah,SA,26.3260,43.9750,
81,cayenne,GF,4.9333,-52.3333,
95,tessalit,ML,20.1986,1.0114,
161,bela,IN,25.9333,81.9833,
195,salalah,OM,17.0151,54.0924,
209,lima,PE,-12.0432,-77.0282,
215,quelimane,MZ,-17.8786,36.8883,
358,machaneng,BW,-23.1862,27.4886,
362,qiongshan,CN,20.0058,110.3542,


In [29]:
params = {"radius": 5000, "types": "lodging", "key": g_key}

for index, row in hotel_df.iterrows():
    lat = row["Lat"]
    lng = row["Lng"]
    
    params["location"] = f"{lat},{lng}"
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    name_address = requests.get(base_url, params=params)
    
    name_address = name_address.json()
    
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["results"][0]["name"]
    except (KeyError, IndexError):
        print("Nearby hotel not found. Skipping...")

hotel_df

NameError: name 'g_key' is not defined

In [30]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [31]:
# Add marker layer ontop of heat map
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# Display figure
fig

NameError: name 'gmaps' is not defined